In [16]:
# import os
import telebot
import yfinance as yf

# API_KEY = os.getenv('API_KEY')
API_KEY = '5336229182:AAFZG8_9m3RaNOlZVZ0oqmPz_xEdk80mXUQ'
bot = telebot.TeleBot(API_KEY)

@bot.message_handler(commands=['greet'])
def greet(message):
  bot.reply_to(message, "Hey! Hows it going?")

@bot.message_handler(commands=['hello'])
def hello(message):
  bot.send_message(message.chat.id, "Hello!")

@bot.message_handler(commands=['wsb'])
def get_stocks(message):
  response = ""
  stocks = ['gme', 'amc', 'nok']
  stock_data = []
  for stock in stocks:
    data = yf.download(tickers=stock, period='2d', interval='1d')

    # reset index to get rid of datetime index, replace it with integeter index
    data = data.reset_index() 
    
    response += f"-----{stock}-----\n"
    stock_data.append([stock])
    columns = ['stock']
    for index, row in data.iterrows():
      stock_position = len(stock_data) - 1
      price = round(row['Close'], 2)
      format_date = row['Date'].strftime('%m/%d')
      response += f"{format_date}: {price}\n"
      stock_data[stock_position].append(price)
      columns.append(format_date)
    print()

  response = f"{columns[0] : <10}{columns[1] : ^10}{columns[2] : >10}\n"
  for row in stock_data:
    response += f"{row[0] : <10}{row[1] : ^10}{row[2] : >10}\n"
  response += "\nStock Data"
  print(response)
  bot.send_message(message.chat.id, response)

def stock_request(message):
  request = message.text.split()
  if len(request) < 2 or request[0].lower() not in "price":
    return False
  else:
    return True

@bot.message_handler(func=stock_request)
def send_price(message):
  request = message.text.split()[1]
  data = yf.download(tickers=request, period='5m', interval='1m')
  if data.size > 0:
    data = data.reset_index()
    data["format_date"] = data['Datetime'].dt.strftime('%m/%d %I:%M %p')
    data.set_index('format_date', inplace=True)
    print(data.to_string())
    bot.send_message(message.chat.id, data['Close'].to_string(header=False))
  else:
    bot.send_message(message.chat.id, "No data!?")

bot.polling()

[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed

stock       04/13        04/14
gme         150.7       148.35
amc         18.53        18.19
nok          5.25         5.21

Stock Data
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- APPL: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
                                Datetime        Open        High         Low       Close   Adj Close  Volume
format_date                                                                                                 
04/14 09:53 AM 2022-04-14 09:53:00-04:00  169.464996  169.500000  169.335007  169.335007  169.335007       0
04/14 09:54 AM 2022-04-14 09:54:00-04:00  169.350006  169.425003  169.300003  169.384995  169.384995  2